In [8]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials
using Plots
using Dates

using Optim
using Random

using TSSOS

Consider simple Lindblad master equation with two dissipators:

$
    \frac{d\rho}{dt} = -\frac{i}{\hbar} [H,\rho(t)] + \mathcal{D}[\rho(t)] = - \frac{i}{\hbar}[H, \rho]+\sum_{\ell=1}^{s-1}\left[J_\ell \rho J_\ell^\dagger - \frac{1}{2}\left\{ J_\ell^\dagger J_\ell, \rho \right\} \right],
$

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} w & 0 \\ 0 & 0
   \end{pmatrix}
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J_1 = \begin{pmatrix} 0 & a \\ 0 & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 & \sqrt{\gamma_1} \\ 0 & 0 
   \end{pmatrix}
$


In [9]:
# Define polynomial variables
@polyvar w α β r ϕ e p

Hˢʸᵐᵇ = [ w   0
          0   0. ]

#Hˢʸᵐᵇ = [ w           α + im* β
#          α + im* β   0.        ]

J₁ˢʸᵐᵇ = [ 0   r
           0   0. ]


function g_objective_6(γᵢ)

    objₑₓ = 0

    for df_trn in train_files # loop over initial states

        ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", γᵢ)
        
        if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
            
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])
        
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ[1:end_train])

        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, [J₁ˢʸᵐᵇ])
        
    end # of files (initial states) loop

    return(objₑₓ)
end

g_objective_6 (generic function with 1 method)

In [10]:

function scaling_poly(p::Polynomial)

    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling)) # 
end

function local_rand_min(p)

    pd = p / maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    num_iterations = 100

    # Initialize the best minimizer and the minimum value
    best_minimizer = nothing
    best_min_value = Inf

    num_of_variables = length(variables(pd))

    for _ in 1:num_iterations
        # Generate a random initial condition
        initial_point = rand(num_of_variables).*250

        # Run local optimization
        result = Optim.optimize(p_scaled, initial_point, BFGS())
        #println(Optim.minimum(result))

        # Update the best minimizer if a better one is found
        if Optim.minimum(result) < best_min_value
            
            best_minimizer = Optim.minimizer(result)
            best_min_value = Optim.minimum(result)
            
        end

    end

    best_minimizer = abs.(best_minimizer) # to make gamma positive

    minimizer_scaled = scale .* best_minimizer

    solution = variables(p_scaled) => minimizer_scaled

end


function tssos_min(p)
    
    ################################################################################################
    #
    #   Try TSSOS on polynomial with scaled variables
    #
    ################################################################################################

    pd = p / maximum(abs.(coefficients(p)))

    # minimize
    # minimizer_scaled_tssos = nothing

    try
        opt,sol,data = tssos_first(pd, variables(pd), QUIET=true, solution=true, newton=true);
        previous_sol = sol

        while ~isnothing(sol)
            previous_sol = sol
            opt,sol,data = tssos_higher!(data; QUIET=true, solution=true);
        end

        global minimizer_tssos = previous_sol
    
    catch
        println("Scaled TSSOS failed")
        global minimizer_scaled_tssos = nothing

    end

    best_solution = variables(p) => minimizer_tssos

end

function tssos_scaled(p::Polynomial)

    pd = p / maximum(abs.(coefficients(p)))

    ################################################################################################
    #
    #   TSSOS on polynomial with scaled variables
    #
    ################################################################################################

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    # minimize
    # minimizer_scaled_tssos = nothing

    try
        opt,sol,data = tssos_first(p_scaled, variables(p), QUIET=true, solution=true, newton=false);
        previous_sol = sol

        while ~isnothing(sol)
            previous_sol = sol
            opt,sol,data = tssos_higher!(data; QUIET=true, solution=true);
        end

        global minimizer_scaled_tssos = scale .* previous_sol
    
    catch
        println("Scaled TSSOS failed")
        global minimizer_scaled_tssos = nothing
    end

    solution = variables(p) => minimizer_scaled_tssos

    return solution
end


tssos_scaled (generic function with 1 method)

In [11]:
data_dir = "../DATA/"
println(data_dir)

models_dir = "../MODELS/"
tests_dir = "../TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

all_files = vcat(dodeca_files, basis_files)
train_files = basis_files 
test_files = dodeca_files;


../DATA/


In [12]:
γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

tests_data_file_name = "POP_LME_1_tssos_scaled_trn4_tst20_"*date_and_time_string * ".h5"

for γᵢ in γ

    println("γ =  "*γᵢ)

    poly = g_objective_6(γᵢ)
    
    sol = tssos_scaled(poly)

    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol)

    omega = subs(w, sol)

    relaxation = subs(r^2, sol)

    omega = subs(w, sol)

    J₁ˢⁱᵈ = subs(J₁ˢʸᵐᵇ, sol)



    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group   

        γ_group["omega"] = convert(Float64, omega)
        γ_group["gamma_relaxation"] = convert(Float64, relaxation)

        
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        #γ_group["J1"] = convert.(ComplexF64, J₁ˢⁱᵈ)
        #γ_group["J2"] = convert.(ComplexF64, J₂ˢⁱᵈ)
        #γ_group["J3"] = convert.(ComplexF64, J₃ˢⁱᵈ)
        #γ_group["J4"] = convert.(ComplexF64, J₄ˢⁱᵈ)
        #γ_group["J5"] = convert.(ComplexF64, J₅ˢⁱᵈ)
        #γ_group["J6"] = convert.(ComplexF64, J₆ˢⁱᵈ)
        
    end

    for df in test_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tˢⁱᵈ, ρˢⁱᵈ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈ), 
                    [ DenseOperator(basis,J₁ˢⁱᵈ)])
                    
        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]                    

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            
        end
                    
        println(minimum(Fᴸᴹᴱₑₓ))
    
    end

    println()
end

println(tests_data_file_name)

γ =  0.079477


*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 2.5593926258622717e-6
Global optimality certified with relative optimality gap 0.002122%!
optimum = 2.5615171305013428e-6
Global optimality certified with relative optimality gap 0.002122%!
No higher TS step of the TSSOS hierarchy!
State_D1 0.9453846724106594


State_D2 

0.9464518597065145
State_D3 0.6444107398572969


State_D4 

...

0.9995608481737658
State_D6 0.9995608481736973
State_D7 0.9976894886273064
State_D8 0.9990714935763321
State_D9 0.9992552340996942
State_D10 0.9994436677278586
State_D11 

0.9991674718917594
State_D12 0.9994878550854149
State_D13 0.9990345415320866
State_D14 0.999484233131915
State_D15 0.9997122953835895
State_D16 0.999712295383576
State_D17 

0.9976894886273064
State_D18 0.9990714935762947
State_D19 0.9994436677278887
State_D20 0.9992552340996161

γ =  251.33


*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.999941649169589
Global optimality certified with relative optimality gap 0.000002%!
optimum = 0.999941648115435
Global optimality certified with relative optimality gap 0.000002%!
No higher TS step of the TSSOS hierarchy!
State_D1 0.996535682115503
State_D2 0.9951092850213458
State_D3 0.9957899341657364
State_D4 0.9944258961366741
State_D5 0.9934629396347414
State_D6 0.9934629396347414
State_D7 0.9996798923766013
State_D8 0.9982727164105653
State_D9 0.994792089645416
State_D10 0.9936328461537857
State_D11 0.995789934165723


State_D12 0.9944258961366741
State_D13 0.996535682115503
State_D14 0.9951092850214078
State_D15 0.993019059479141
State_D16 0.993019059479094
State_D17 0.9996798923766137
State_D18 0.9982727164105617
State_D19 0.9936328461537857
State_D20 0.9947920896453086

POP_LME_1_tssos_scaled_trn4_tst20_2024-Jan-16_at_14-48.h5
